<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#추천시스템" data-toc-modified-id="추천시스템-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>추천시스템</a></span><ul class="toc-item"><li><span><a href="#비슷한-유저-찾기" data-toc-modified-id="비슷한-유저-찾기-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>비슷한 유저 찾기</a></span></li><li><span><a href="#비슷한-유저가-본-영화-추천-&amp;-평점순-우선순위" data-toc-modified-id="비슷한-유저가-본-영화-추천-&amp;-평점순-우선순위-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>비슷한 유저가 본 영화 추천 &amp; 평점순 우선순위</a></span></li><li><span><a href="#추천-영화-합치기" data-toc-modified-id="추천-영화-합치기-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>추천 영화 합치기</a></span></li><li><span><a href="#추천-영화-이름-보기" data-toc-modified-id="추천-영화-이름-보기-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>추천 영화 이름 보기</a></span></li><li><span><a href="#함수-모아보기" data-toc-modified-id="함수-모아보기-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>함수 모아보기</a></span></li></ul></li><li><span><a href="#folium으로-가짜-코로나19-지도-만들기" data-toc-modified-id="folium으로-가짜-코로나19-지도-만들기-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>folium으로 가짜 코로나19 지도 만들기</a></span><ul class="toc-item"><li><span><a href="#DataFrame에서-for문-쓰는-방법-4가지" data-toc-modified-id="DataFrame에서-for문-쓰는-방법-4가지-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DataFrame에서 for문 쓰는 방법 4가지</a></span></li></ul></li></ul></div>

# [AI 오픈 이노베이션] 기본 저녁반<a class="tocSkip">
GitHub link: <a href="https://github.com/realblack0/ai_lecture_fundamental">here</a>  
E-Mail: realblack0@gmail.com

## 추천시스템

이번 섹션의 실습 데이터는 LMS의 3월 11일 첨부자료에 있습니다. (movie-master.zip)

### 비슷한 유저 찾기

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv("movie/ratings.csv")

In [3]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


먼저, 원본 데이터를 카피해둔다.

In [4]:
ratings_raw = ratings.copy()

timestamp는 영화 평점을 매긴 시간이다.  
어떻게 추천할지 전략에 따라서 활용할 수 있다. (예: 오래된 평점은 무시하고, 최신 평점은 가중치를 더 주는 방법)  
이번 추천시스템 실습에서는 사용하지 않을 것이므로 삭제한다.

In [5]:
recommendation_1 = ratings.drop(columns="timestamp")

user_id를 인덱스로, movie_id를 열로 데이터를 가공한다.  
결측값은 0으로 채운다.  
데이터를 가공할 때마다 새로 할당하면 이전 단계로 되돌아가기 좋다.

In [6]:
recommendation_2 = recommendation_1.set_index(["user_id", "movie_id"]).unstack().fillna(0)

전체 데이터의 양이 꽤 크기 때문에 상관관계를 계산하는데 시간이 오래 걸린다.  
실습 시간을 단축하기 위해서 데이터를 일부만 사용한다.(현실 문제에서는 더 좋은 컴퓨터를 마련해야한다.)

In [7]:
recommendation_3 = recommendation_2.iloc[:1000, :1000] # 실습 시간 문제

`T`는 인덱스와 컬럼을 전치하는 메소드이다.  
`corr`은 컬럼끼리의 상관관계를 구하는 메소드이다.  
유저간의 상관관계를 분석할 것이므로, user_id가 컬럼이 되도록 전치한 후 `corr`하였다.

In [8]:
recommendation_4 = recommendation_3.T.corr()

4번 유저(user_id=4)와 다른 유저들간의 상관관계를 뽑아올 수 있다.

In [9]:
recommendation_4.iloc[3] # iloc이므로 인덱스 기준

user_id
1       0.128041
2       0.117076
3       0.421361
4       1.000000
5      -0.012701
          ...   
996     0.235446
997     0.146728
998    -0.011323
999     0.130853
1000    0.370799
Name: 4, Length: 1000, dtype: float64

4번 유저와 상관관계가 높은 유저를 찾아보자. (상관관계가 높다 = 비슷하다)  
`sort_values`는 값을 순서대로 정렬해준다. `ascending=False` 옵션은 내림차순으로 정렬한다.  
상위 5개 데이터만 슬라이싱한다.

In [10]:
recommendation_4.iloc[3].sort_values(ascending=False)[:5]

user_id
4      1.000000
160    0.794130
747    0.726089
763    0.623551
248    0.609241
Name: 4, dtype: float64

`nlargest`는 가장 큰 값을 n개 구하는 메소드이다.  
위 코드를 메소드로 간단히 대체할 수 있다.

In [11]:
recommendation_4.iloc[3].nlargest() # 기본값 n=5

user_id
4      1.000000
160    0.794130
747    0.726089
763    0.623551
248    0.609241
Name: 4, dtype: float64

In [12]:
recommendation_4.iloc[3].nlargest(3) # n=3일 때

user_id
4      1.000000
160    0.794130
747    0.726089
Name: 4, dtype: float64

자기자신과의 상관관계는 1이므로 상관관계 내림차순으로 정렬하면 첫번째는 항상 자기자신이다.  
4번 유저와 비슷한 다른 유저들을 찾는게 목적이므로, 슬라이싱으로 자기자신을 제외한다.

In [13]:
recommendation_4.loc[4].nlargest()[1:] # loc은 이름 기준

user_id
160    0.794130
747    0.726089
763    0.623551
248    0.609241
Name: 4, dtype: float64

비슷한 유저를 찾는 일은 추천할 때마다 반복할 것이므로, 함수로 만들어둔다.

In [14]:
def my_corr(my_id, n):
    return recommendation_4.loc[my_id].nlargest(n+1)[1:]

In [15]:
my_corr(4, 5) # 4번 유저와 가장 비슷한 유저 5명

user_id
160    0.794130
747    0.726089
763    0.623551
248    0.609241
206    0.603656
Name: 4, dtype: float64

### 비슷한 유저가 본 영화 추천 & 평점순 우선순위

4번 유저와 가장 비슷한 유저는 160번이었다.  
160번이 본 영화는 다음과 같이 boolean indexing할 수 있다.

In [16]:
recommendation_1[recommendation_1.user_id==160]

,user_id,movie_id,rating
21827,160,648,2
21828,160,3793,3
21829,160,3798,3
21830,160,1282,4
21831,160,1286,5
21832,160,2160,3
21833,160,1387,3
21834,160,1396,4
21835,160,1544,3
21836,160,3185,3


이 중에서 movie_id열만 뽑아서 numpy array로 변환한다.  
4번 유저와 160번 유저는 (영화 취향이) 비슷하므로, 160번 유저가 본 영화를 4번 유저에게 추천할 수 있다.

In [17]:
recommendation_1[recommendation_1.user_id==160]["movie_id"].values

array([ 648, 3793, 3798, 1282, 1286, 2160, 1387, 1396, 1544, 3185, 3408,
       3510,  260, 2858, 2791, 3598, 1210,  480, 3916, 1097], dtype=int64)

데이터를 set(집합)으로 타입을 변환할 수 있다.  
set은 중복성 검사할때 유용하다.

In [18]:
set(recommendation_1[recommendation_1.user_id==160]["movie_id"].values)

{260,
 480,
 648,
 1097,
 1210,
 1282,
 1286,
 1387,
 1396,
 1544,
 2160,
 2791,
 2858,
 3185,
 3408,
 3510,
 3598,
 3793,
 3798,
 3916}

160번 유저가 본 영화 목록에서, 4번 유저가 본 영화를 제외하고 추천하기 위해서 차집합을 이용한다.  
다음은 추천할 영화들이다.

In [19]:
recom_movie_id = set(recommendation_1[recommendation_1.user_id == 160]["movie_id"].values) - \
                     set(recommendation_1[recommendation_1.user_id == 4]["movie_id"].values)

In [20]:
recom_movie_id

{648,
 1282,
 1286,
 1396,
 1544,
 2160,
 2791,
 2858,
 3185,
 3408,
 3510,
 3598,
 3793,
 3798,
 3916}

현재는 추천할 영화 목록만 있는 상태이다. 추천에는 우선순위가 필요하다. 영화 평점으로 추천 우선순위를 구해보자.  
추천 대상이 된 영화들의 평점을 `isin` 메소드와 boolean indexing으로 가져온다.

In [21]:
recommendation_1[recommendation_1.movie_id.isin(recom_movie_id)]

,user_id,movie_id,rating
3,1,3408,4
15,1,2791,4
58,2,648,4
87,2,1544,4
105,2,2858,4
...,...,...,...
999582,6037,2160,5
999833,6039,1282,3
999859,6039,2791,4
999938,6040,2858,4


추천 대상은 160번 유저가 본 영화들이기 때문에, 평점의 공정성을 위해서 160번의 평점은 제외하자.  
boolean indexing할때 조건이 여러개이면 괄호로 묶어준다.

In [22]:
recommendation_5 = recommendation_1[recommendation_1.movie_id.isin(recom_movie_id) & (recommendation_1.user_id!=160)]

집계함수 `groupby`로 영화별 평균 평점을 구한 다음, 평균 평점이 가장 높은 5개 영화를 우선 추천한다.

In [23]:
recommendation_5.groupby("movie_id")['rating'].mean().nlargest(5)

movie_id
2858    4.317479
2791    3.971676
3916    3.922018
1282    3.904459
3408    3.863775
Name: rating, dtype: float64

비슷한 유저가 본 영화를 평점 순으로 추천하는 과정도 반복할 것이므로 함수로 만든다.

In [25]:
def recom_movie(my_id, user_id, count):
    recom_movie_id = set(recommendation_1[recommendation_1.user_id == user_id]["movie_id"].values) - \
        set(recommendation_1[recommendation_1.user_id == my_id]["movie_id"].values)
    recommendation_5 = recommendation_1[recommendation_1.movie_id.isin(recom_movie_id) &  
                                        (recommendation_1.user_id != user_id)] 
    return recommendation_5.groupby("movie_id")['rating'].mean().nlargest(count).index

In [26]:
recom_movie(4, 160, 8) # 4번 유저에게 160번 유저가 본 영화를 추천하는데
                       # 평점 순으로 8개 추천.

Int64Index([2858, 2791, 3916, 1282, 3408, 3793, 2160, 3510], dtype='int64', name='movie_id')

### 추천 영화 합치기

앞서 만든 `my_corr` 함수로 비슷한 유저들을 찾은 뒤,  
비슷한 유저가 본 영화 추천 목록을 temp에 저장한다.

In [27]:
temp = []
for i in my_corr(4, 5).index: 
    temp.append(recom_movie(4, i, 5))

In [28]:
temp

[Int64Index([2858, 2791, 3916, 1282, 3408], dtype='int64', name='movie_id'),
 Int64Index([2571, 3897, 1200, 3949, 3578], dtype='int64', name='movie_id'),
 Int64Index([1136, 1213, 541, 1225, 919], dtype='int64', name='movie_id'),
 Int64Index([2571, 1197, 110, 1242, 1222], dtype='int64', name='movie_id'),
 Int64Index([1193, 2571, 3897, 1200, 924], dtype='int64', name='movie_id')]

각각의 추천 목록을 temp2에 합친다.

In [29]:
temp2 = [] # accumulation
for i in temp:
    for j in i:
        temp2.append(j)

In [30]:
temp2

[2858,
 2791,
 3916,
 1282,
 3408,
 2571,
 3897,
 1200,
 3949,
 3578,
 1136,
 1213,
 541,
 1225,
 919,
 2571,
 1197,
 110,
 1242,
 1222,
 1193,
 2571,
 3897,
 1200,
 924]

`set`을 이용해서 중복되는 추천 영화를 제거한다.

In [31]:
set(temp2)

{110,
 541,
 919,
 924,
 1136,
 1193,
 1197,
 1200,
 1213,
 1222,
 1225,
 1242,
 1282,
 2571,
 2791,
 2858,
 3408,
 3578,
 3897,
 3916,
 3949}

합쳐진 추천 영화 목록에서, 다시 평점 순으로 추천 우선순위를 정하여 최종 추천한다.

In [32]:
temp3 = recommendation_1[recommendation_1.movie_id.isin(set(temp2))] 

In [33]:
temp4 = temp3.groupby("movie_id").rating.mean()

In [34]:
temp4.nlargest(5) # 최종 추천 영화

movie_id
1193    4.390725
1136    4.335210
2858    4.317386
2571    4.315830
1197    4.303710
Name: rating, dtype: float64

위 과정을 함수로 만들면 다음과 같다.

In [35]:
def final_recom_movie(my_id, similarity_count=5, recom_count=5):
    temp = []
    for i in my_corr(my_id, similarity_count).index:
        temp.append(recom_movie(my_id, i, recom_count))
    temp2 = []
    for i in temp:
        for j in i:
            temp2.append(j)
    temp3 = recommendation_1[recommendation_1.movie_id.isin(set(temp2))]
    temp4 = temp3.groupby("movie_id").rating.mean()
    return temp4.nlargest(recom_count).index

In [36]:
final_recom_movie(4) # 4번 유저에 대한 추천 영화

Int64Index([1193, 1136, 2858, 2571, 1197], dtype='int64', name='movie_id')

### 추천 영화 이름 보기

In [37]:
final_recom_movie(6)

Int64Index([318, 527, 1148, 1198, 260], dtype='int64', name='movie_id')

In [38]:
movie_id = final_recom_movie(6)

In [39]:
movie = pd.read_csv("movie/movies.csv", encoding="latin1")

In [40]:
movie[movie.movie_id.isin(movie_id)]

,movie_id,movie,genres
257,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
315,318,"Shawshank Redemption, The (1994)",Drama
523,527,Schindler's List (1993),Drama|War
1132,1148,"Wrong Trousers, The (1993)",Animation|Comedy
1180,1198,Raiders of the Lost Ark (1981),Action|Adventure


### 함수 모아보기

In [41]:
def my_corr(my_id, n):
    return recommendation_4.loc[my_id].nlargest(n+1)[1:]

In [47]:
def recom_movie(my_id, user_id, count):
    recom_movie_id = set(recommendation_1[recommendation_1.user_id == user_id]["movie_id"].values) - \
        set(recommendation_1[recommendation_1.user_id == my_id]["movie_id"].values)
    recommendation_5 = recommendation_1[recommendation_1.movie_id.isin(recom_movie_id) & 
                                        (recommendation_1.user_id != user_id)] 
    return recommendation_5.groupby("movie_id")['rating'].mean().nlargest(count).index

In [48]:
def final_recom_movie(my_id, similarity_count=5, recom_count=5):
    temp = []
    for i in my_corr(my_id, similarity_count).index:
        temp.append(recom_movie(my_id, i, recom_count))
    temp2 = []
    for i in temp:
        for j in i:
            temp2.append(j)
    temp3 = recommendation_1[recommendation_1.movie_id.isin(set(temp2))]
    temp4 = temp3.groupby("movie_id").rating.mean()
    return temp4.nlargest(recom_count).index

## folium으로 가짜 코로나19 지도 만들기

folium을 이용해서 최근 이슈가 되는 코로나 지도를 만들 수 있다. 

※ folium의 기본 사용법은 LMS의 3월 12일 첨부자료에 있습니다. (Geo Data.zip)  
※ 실습 데이터는 공공데이터 포털의 상권(상가)정보 를 이용하였습니다. (다운로드 후 이름 변경)

In [49]:
import folium
import pandas as pd
import numpy as np

보안설정이 높게 되어 있으면 빈 화면으로 보일 수 있다.(특히 익스플로러)  
PC의 보안 수준을 낮추거나 firefox를 이용하면 지도가 정상 출력된다.

In [50]:
m = folium.Map([37.5665, 126.9780], zoom_start=15)

In [51]:
m

상권 정보 데이터에 실제 위도/경도 데이터가 있으므로, 이를 활용한다.

In [52]:
a = pd.read_csv("a.csv", sep="|")

In [53]:
a.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [54]:
b = a[["경도", "위도"]]

In [57]:
b

,경도,위도,감염자
0,127.061026,37.493922,91
1,127.047883,37.505675,85
2,126.899220,37.471711,62
3,128.980455,35.159774,86
4,127.104071,37.500249,51
...,...,...,...
573675,126.955059,37.548618,54
573676,127.054615,37.545853,77
573677,127.024102,37.506202,68
573678,126.924863,37.525172,75


코로나 바이러스의 감염자 수를 랜덤하게 생성한다.

In [58]:
b["감염자"] = 0 # 감염자 열을 생성
b["감염자"] = b["감염자"].map(lambda x:np.random.randint(50, 100)) # 감염자 열의 값을 50~100 사이 정수로 랜덤 생성

C:\Users\JINHYO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\JINHYO\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


20개 정도만 잘라서 사용한다.

In [59]:
c = b.iloc[:20, :]

In [60]:
c

,경도,위도,감염자
0,127.061026,37.493922,52
1,127.047883,37.505675,62
2,126.899220,37.471711,91
3,128.980455,35.159774,63
4,127.104071,37.500249,93
5,127.102490,37.515149,90
6,127.088387,37.549245,82
7,126.809957,37.562013,93
8,127.010861,37.552153,61
9,126.836078,37.538927,60


### DataFrame에서 for문 쓰는 방법 4가지

DataFrame을 그냥 for문에 넣으면 컬럼 이름을 하나씩 반환한다.

In [61]:
for i in c:
    print(i)

경도
위도
감염자


`iteritems`는 컬럼을 하나씩 반환한다. (Series로)

In [62]:
for i in c.iteritems():
    print(i)

('경도', 0     127.061026
1     127.047883
2     126.899220
3     128.980455
4     127.104071
5     127.102490
6     127.088387
7     126.809957
8     127.010861
9     126.836078
10    127.014217
11    126.886859
12    127.031702
13    126.880898
14    127.028726
15    126.897710
16    126.910288
17    127.043189
18    127.071184
19    127.076812
Name: 경도, dtype: float64)
('위도', 0     37.493922
1     37.505675
2     37.471711
3     35.159774
4     37.500249
5     37.515149
6     37.549245
7     37.562013
8     37.552153
9     37.538927
10    37.492388
11    37.480390
12    37.588485
13    37.504067
14    37.608392
15    37.503693
16    37.579029
17    37.580073
18    37.552103
19    37.545461
Name: 위도, dtype: float64)
('감염자', 0     52
1     62
2     91
3     63
4     93
5     90
6     82
7     93
8     61
9     60
10    77
11    55
12    70
13    53
14    67
15    68
16    71
17    74
18    59
19    72
Name: 감염자, dtype: int64)


`iterrows`는 로우를 하나씩 반환한다.

In [63]:
for i in c.iterrows(): 
    print(i)

(0, 경도     127.061026
위도      37.493922
감염자     52.000000
Name: 0, dtype: float64)
(1, 경도     127.047883
위도      37.505675
감염자     62.000000
Name: 1, dtype: float64)
(2, 경도     126.899220
위도      37.471711
감염자     91.000000
Name: 2, dtype: float64)
(3, 경도     128.980455
위도      35.159774
감염자     63.000000
Name: 3, dtype: float64)
(4, 경도     127.104071
위도      37.500249
감염자     93.000000
Name: 4, dtype: float64)
(5, 경도     127.102490
위도      37.515149
감염자     90.000000
Name: 5, dtype: float64)
(6, 경도     127.088387
위도      37.549245
감염자     82.000000
Name: 6, dtype: float64)
(7, 경도     126.809957
위도      37.562013
감염자     93.000000
Name: 7, dtype: float64)
(8, 경도     127.010861
위도      37.552153
감염자     61.000000
Name: 8, dtype: float64)
(9, 경도     126.836078
위도      37.538927
감염자     60.000000
Name: 9, dtype: float64)
(10, 경도     127.014217
위도      37.492388
감염자     77.000000
Name: 10, dtype: float64)
(11, 경도     126.886859
위도      37.480390
감염자     55.000000
Name: 11, dtype: float64)


`itertuples`는 로우를 튜플로 반환한다. (namedtuple과 비슷함)

In [64]:
for i in c.itertuples():
    print(i)

Pandas(Index=0, 경도=127.061026050567, 위도=37.4939223326479, 감염자=52)
Pandas(Index=1, 경도=127.047883157395, 위도=37.505675040057994, 감염자=62)
Pandas(Index=2, 경도=126.89922037068199, 위도=37.4717112331268, 감염자=91)
Pandas(Index=3, 경도=128.98045527926598, 위도=35.1597741758499, 감염자=63)
Pandas(Index=4, 경도=127.104070564081, 위도=37.5002486691845, 감염자=93)
Pandas(Index=5, 경도=127.102489784096, 위도=37.515149192048504, 감염자=90)
Pandas(Index=6, 경도=127.088387455895, 위도=37.549245207781404, 감염자=82)
Pandas(Index=7, 경도=126.80995747255501, 위도=37.562012692529294, 감염자=93)
Pandas(Index=8, 경도=127.010861306352, 위도=37.5521533135816, 감염자=61)
Pandas(Index=9, 경도=126.83607800821699, 위도=37.538927347810294, 감염자=60)
Pandas(Index=10, 경도=127.014217033954, 위도=37.492387659064704, 감염자=77)
Pandas(Index=11, 경도=126.886859445485, 위도=37.48038965624, 감염자=55)
Pandas(Index=12, 경도=127.031701809695, 위도=37.5884849860916, 감염자=70)
Pandas(Index=13, 경도=126.88089820110399, 위도=37.5040668669735, 감염자=53)
Pandas(Index=14, 경도=127.028725560472, 위도=37.60839154

`Circle` 클래스로 지도에 원을 그릴 수 있다.  
`Circle`을 이용해서 코로나 지도를 그릴 것이다.

In [65]:
m4 = folium.Map(location=[37.5665, 126.9780], tiles='Stamen Toner', zoom_start=15)

folium.Circle([37.5665, 126.9780], radius=100).add_to(m4) 

m4

앞서 만든 가짜 코로나 데이터를 for문으로 지도에 표시한다.

In [66]:
m4 = folium.Map(location=[37.5665, 126.9780], tiles='Stamen Terrain', zoom_start=12)

for i in c.itertuples(): # 튜플안에서 데이터의 순서 조심
    folium.Circle(radius=i[3]*20,                    # 감염자수는 3번째에 있음 / *20 은 원 크기 조절
                  location=[i[2], i[1]],             # 위도는 2번째, 경도는 3번째에 있음
                  popup="patients: " + str(i[3]),    # 원을 클릭하면 문구가 뜸
                  color="#C4302B",                   # 원 색상은 빨간색
                  fill=True,                         # 원 안을 채움
                  fill_color="#C4302B").add_to(m4)   # 원 안의 색은 빨간색

m4